In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import torch

from umap.umap_ import UMAP
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation
from tqdm import tqdm

In [222]:
df = pd.read_csv('./data/applic.csv')

In [227]:
print(df['text'][4])


20230731_12345
Китайский документ 1-10trans.txt
Глава I Уведомление о торгах
Место проведения торгов: Янцзы, провинция Гуандун
Условия торгов
Проект &quot; Яньцзян &quot; , проект &quot; Яньцзян &quot; , проект &quot; Модернизация модуля динамического преобразования оборудования &quot; Яньцзян &quot; (второе тендерное предложение) &quot; (номер тендера: CGN-
202305150006-N1 был утвержден, проект финансируется за счет самофинансирования и реализован, а тендер - за счет компании Jiangian.
В рамках этого проекта уже имеются условия для проведения торгов и в настоящее время проводятся открытые торги.
II. ОБЩИЙ ОБЗОР ПРОЕКТОВ И ОГРАНИЧЕНИЙ
Общая информация по проекту: установка RX на АЭС &lt; &lt; Янцзян &gt; &gt; (DMR001PR) и установка 1/2/5/6
(DMW115PA) - акселераторы, ретрансформаторы или трансмиссионные серии ABB серии ACS800, которые были использованы в связи с прекращением производства оборудования марки.
Перестройка.
В таблице ниже приводится перечень моделей и количества оригинальн

In [6]:
class VectorStore:
    def __init__(self, model, size):
        self.documents = []
        self.embeddings = np.empty(size)
        self.model = model  

    def add_to_store(self, document):
        self.documents.append(document)
        embedding = self.model.encode(document, convert_to_numpy=True, normalize_embeddings=True, show_progress_bar=False, batch_size=32)
        self.embeddings = np.vstack((self.embeddings, embedding))

In [7]:
model_name = '../models/embaas_sentence-transformers-e5-large-v2/'

In [10]:
device = torch.device("mps")
device

device(type='mps')

In [11]:
model = SentenceTransformer(model_name, device=device)
model[1].pooling_mode_mean_tokens = False
model[1].pooling_mode_cls_token = True
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [12]:
store = VectorStore(model, size=(0, model[1].word_embedding_dimension)) 
for j in tqdm(range(df.shape[0])):
    store.add_to_store(df['text'][j])

100%|██████████| 1429/1429 [01:30<00:00, 15.82it/s]


In [13]:
store.embeddings.shape

(1429, 1024)

In [14]:
tsne = TSNE(n_components=2, random_state=42)
projections = tsne.fit_transform(store.embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
kmeans = KMeans(n_clusters=15)
kmeans.fit(projections)
labels = kmeans.labels_
data_labeled = np.column_stack((projections, labels))
df = pd.DataFrame(data_labeled, columns=['X', 'Y', 'labels'])
df['labels'] = df['labels'].astype('str')
centroids = kmeans.cluster_centers_

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [43]:
def build_scatter_2d(data, projections, title):
    fig = px.scatter(projections,
                x=0, 
                y=1,
                color=data['labels'], 
                labels={'color': 'category'},
                template='plotly_dark',
                height=700,
                )

    fig.update_layout(
                title={'text': title,
                        'y':0.95,
                        'x':0.5},
                showlegend=True)

    fig.show()

In [18]:
build_scatter_2d(df, projections, centroids, 'K-means')

In [19]:
umap_2d = UMAP(n_components=2, init='random', random_state=42)
projections = umap_2d.fit_transform(store.embeddings)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [54]:
umap_model = UMAP(n_neighbors=5, min_dist=0.3, metric='correlation')
projections = umap_model.fit_transform(store.embeddings)

In [57]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(projections)
labels = kmeans.labels_
data_labeled = np.column_stack((projections, labels))
df = pd.DataFrame(data_labeled, columns=['X', 'Y', 'labels'])
df['labels'] = df['labels'].astype('str')


/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [58]:
build_scatter_2d(df, projections, 'K-means')

In [98]:
clustering = AffinityPropagation(random_state=42, max_iter=300, damping= 0.98, convergence_iter=15).fit(projections)

In [99]:
labels = clustering.labels_
data_labeled = np.column_stack((projections, labels))
df = pd.DataFrame(data_labeled, columns=['X', 'Y', 'labels'])
df['labels'] = df['labels'].astype('str')

In [100]:
build_scatter_2d(df, projections, 'Afiinity Propagation')

In [217]:
dbscan = DBSCAN(eps=0.3, min_samples=5)

In [218]:
clusters = dbscan.fit_predict(projections)

In [219]:
labels = clusters
data_labeled = np.column_stack((projections, labels))
df = pd.DataFrame(data_labeled, columns=['X', 'Y', 'labels'])
df['labels'] = df['labels'].astype('str')

In [220]:
build_scatter_2d(df, projections, 'DBSCAN')